In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.grid_search import GridSearchCV  # Perforing grid search
from sklearn.model_selection import train_test_split

train_data = pd.read_csv('train_last.csv')   # 读取数据

/home/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
train_data.columns


Index(['飞机编号', '航空公司', '时间段', '机场', '航班编号', '天气', '常见特情内容', '平均气温', '特情内容数量',
       '计划飞行时间', 'y'],
      dtype='object')

In [3]:
train_data.head()

,飞机编号,航空公司,时间段,机场,航班编号,天气,常见特情内容,平均气温,特情内容数量,计划飞行时间,y
0,2124.0,CZ,早上,AAT,CZ6844,晴,0,13.0,0,1.083333,False
1,102.0,GS,早上,AAT,GS7556,晴,0,13.0,0,1.000000,False
2,1214.0,CZ,晚上,AAT,CZ6842,晴,0,13.0,0,1.083333,False
3,1992.0,GS,晚上,AAT,GS7582,晴,0,13.0,0,1.000000,False
4,NaN,GS,凌晨,AAT,GS7582A,晴,0,15.5,0,1.000000,False


In [4]:
train_data = train_data.dropna()

In [5]:
train_data.head()

,飞机编号,航空公司,时间段,机场,航班编号,天气,常见特情内容,平均气温,特情内容数量,计划飞行时间,y
0,2124.0,CZ,早上,AAT,CZ6844,晴,0,13.0,0,1.083333,False
1,102.0,GS,早上,AAT,GS7556,晴,0,13.0,0,1.000000,False
2,1214.0,CZ,晚上,AAT,CZ6842,晴,0,13.0,0,1.083333,False
3,1992.0,GS,晚上,AAT,GS7582,晴,0,13.0,0,1.000000,False
5,276.0,CZ,早上,AAT,CZ6844,晴,0,15.5,0,1.000000,False


In [6]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train_data = train_data.apply(preprocessing.LabelEncoder().fit_transform)#把列做label encoding

In [7]:
train_data.head()

,飞机编号,航空公司,时间段,机场,航班编号,天气,常见特情内容,平均气温,特情内容数量,计划飞行时间,y
0,2123,19,3,0,8176,112,0,95,0,46,0
1,101,33,3,0,11768,112,0,95,0,42,0
2,1213,19,4,0,8173,112,0,95,0,46,0
3,1991,33,4,0,11823,112,0,95,0,42,0
5,275,19,3,0,8176,112,0,100,0,42,0


In [8]:
#隐藏警告
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.grid_search import GridSearchCV  # Perforing grid search
from sklearn.model_selection import train_test_split

#train_data = pd.read_csv('train_last.csv')   # 读取数据
y = train_data.pop('y').values   # 用pop方式将训练数据中的标签值y取出来，作为训练目标，这里的‘30’是标签的列名
col = train_data.columns   
x = train_data[col].values  # 剩下的列作为训练数据
train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=0.333, random_state=0)   # 分训练集和验证集
train = lgb.Dataset(train_x, train_y)
valid = lgb.Dataset(valid_x, valid_y, reference=train)


parameters = {
              'max_depth': [15, 20, 25, 30, 35],
              'learning_rate': [0.01, 0.02, 0.05, 0.1, 0.15],
              'feature_fraction': [0.6, 0.7, 0.8, 0.9, 0.95],
              'bagging_fraction': [0.6, 0.7, 0.8, 0.9, 0.95],
              'bagging_freq': [2, 4, 5, 6, 8],
              'lambda_l1': [0, 0.1, 0.4, 0.5, 0.6],
              'lambda_l2': [0, 10, 15, 35, 40],
              'cat_smooth': [1, 10, 15, 20, 35]
}
gbm = lgb.LGBMClassifier(boosting_type='gbdt',
                         objective = 'binary',
                         metric = 'auc',
                         verbose = 0,
                         learning_rate = 0.01,
                         num_leaves = 35,
                         feature_fraction=0.8,
                         bagging_fraction= 0.9,
                         bagging_freq= 8,
                         lambda_l1= 0.6,
                         lambda_l2= 0)
# 有了gridsearch我们便不需要fit函数
gsearch = GridSearchCV(gbm, param_grid=parameters, scoring='accuracy', cv=3)
gsearch.fit(train_x, train_y)

print("Best score: %0.3f" % gsearch.best_score_)
print("Best parameters set:")
best_parameters = gsearch.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

